In [11]:
import numpy as np
import os
import json
import pickle as pkl

# Get file path

In [12]:
def document_path_list(mvt_path):
    method_dir = ["blazepose","openpose", "kinect", "vicon"]
    group_name = "group3"
    group_path = os.path.join(mvt_path,group_name)
#         print(group_path)
    dic_method = {}
    for method_name in method_dir:
        method_path = os.path.join(group_path,method_name)
        json_dir = os.listdir(method_path)
        
        file_read_list = []
        for json_name in json_dir:
            file_read_path = os.path.join(method_path,json_name)
            file_read_list.append(file_read_path)
#                 print(file_read_path)
        dic_method[method_name] = sorted(file_read_list)
    return dic_method

# Convert frame to Vicon dic

In [13]:
joint = ["Right_Forearm", "Left_Forearm", "Right_Arm", "Left_Arm", "Chest",
            "Right_Tigh", "Left_Tigh", "Right_Shoulder", "Left_Shoulder",
            "Right_Hand", "Left_Hand", "Right_Foot", "Left_Foot",
            "Hips", "Head", "Right_Tibia", "Left_Tibia"]
def vicon_frame_to_dic(line):
    joint = ["Right_Forearm", "Left_Forearm", "Right_Arm", "Left_Arm", "Chest",
            "Right_Tigh", "Left_Tigh", "Right_Shoulder", "Left_Shoulder",
            "Right_Hand", "Left_Hand", "Right_Foot", "Left_Foot",
            "Hips", "Head", "Right_Tibia", "Left_Tibia"]
    data = line.split(" ")[:-1]
    vicon_dic = {}
    for num in range(len(joint)):
        vicon_dic[joint[num]] = np.array([float(data[num*7]), float(data[num*7+1]), float(data[num*7+2])])
    return vicon_dic

In [14]:
def open_frame_to_vicon(open_dic):
    open_list = ["Head","mShoulder",
                "rShoulder","rElbow","rWrist","lShoulder","lElbow","lWrist",
                "rHip","rKnee","rAnkle","lHip","lKnee","lAnkle"]
    for key in open_list:
        if key not in open_dic.keys():
            open_dic[key] = np.array([np.nan, np.nan])
    new_open_dic = {}
    #spine
    new_open_dic["Head"] = np.array(open_dic["Head"])
    new_open_dic["Chest"] = (np.array(open_dic["mShoulder"])+(np.array(open_dic["rHip"])+np.array(open_dic["lHip"]))/2)/2
    new_open_dic["Hips"] = (np.array(open_dic["rHip"])+np.array(open_dic["lHip"]))/2
    #left arm
    new_open_dic["Left_Shoulder"] = np.array(open_dic["lShoulder"])
    new_open_dic["Left_Arm"] = (np.array(open_dic["lShoulder"])+np.array(open_dic["lElbow"]))/2
    new_open_dic["Left_Forearm"] = (np.array(open_dic["lElbow"])+np.array(open_dic["lWrist"]))/2
    new_open_dic["Left_Hand"] = np.array([np.nan, np.nan])
    #right arm
    new_open_dic["Right_Shoulder"] = np.array(open_dic["rShoulder"])
    new_open_dic["Right_Arm"] = (np.array(open_dic["rShoulder"])+np.array(open_dic["rElbow"]))/2
    new_open_dic["Right_Forearm"] = (np.array(open_dic["rElbow"])+np.array(open_dic["rWrist"]))/2
    new_open_dic["Right_Hand"] = np.array([np.nan, np.nan])
    #left leg
    new_open_dic["Left_Tigh"] = (np.array(open_dic["lHip"])+np.array(open_dic["lKnee"]))/2
    new_open_dic["Left_Tibia"] = (np.array(open_dic["lKnee"])+np.array(open_dic["lAnkle"]))/2
    new_open_dic["Left_Foot"] = np.array([np.nan, np.nan])
    #right leg
    new_open_dic["Right_Tigh"] = (np.array(open_dic["rHip"])+np.array(open_dic["rKnee"]))/2
    new_open_dic["Right_Tibia"] = (np.array(open_dic["rKnee"])+np.array(open_dic["rAnkle"]))/2
    new_open_dic["Right_Foot"] = np.array([np.nan, np.nan])
    return new_open_dic

In [15]:
def blaze_frame_to_vicon(blaze_dic):
    new_blaze_dic = {}
    #spine
    new_blaze_dic["Head"] = np.array(blaze_dic["Nose"])
    new_blaze_dic["Chest"] = ((np.array(blaze_dic["Left_shoulder"])+np.array(blaze_dic["Right_shoulder"]))/2+(np.array(blaze_dic["Left_hip"])+np.array(blaze_dic["Right_hip"]))/2)/2
    new_blaze_dic["Hips"] = (np.array(blaze_dic["Left_hip"])+np.array(blaze_dic["Right_hip"]))/2
    #left arm
    new_blaze_dic["Left_Shoulder"] = np.array(blaze_dic["Left_shoulder"])
    new_blaze_dic["Left_Arm"] = (np.array(blaze_dic["Left_shoulder"])+np.array(blaze_dic["Left_elbow"]))/2
    new_blaze_dic["Left_Forearm"] = (np.array(blaze_dic["Left_elbow"])+np.array(blaze_dic["Left_wrist"]))/2
    new_blaze_dic["Left_Hand"] = (np.array(blaze_dic["Left_pinky"])+np.array(blaze_dic["Left_index"]))/2
    #right arm
    new_blaze_dic["Right_Shoulder"] = np.array(blaze_dic["Right_shoulder"])
    new_blaze_dic["Right_Arm"] = (np.array(blaze_dic["Right_shoulder"])+np.array(blaze_dic["Right_elbow"]))/2
    new_blaze_dic["Right_Forearm"] = (np.array(blaze_dic["Right_elbow"])+np.array(blaze_dic["Right_wrist"]))/2
    new_blaze_dic["Right_Hand"] = (np.array(blaze_dic["Right_pinky"])+np.array(blaze_dic["Right_index"]))/2
    #left leg
    new_blaze_dic["Left_Tigh"] = (np.array(blaze_dic["Left_hip"])+np.array(blaze_dic["Left_knee"]))/2
    new_blaze_dic["Left_Tibia"] = (np.array(blaze_dic["Left_knee"])+np.array(blaze_dic["Left_ankle"]))/2
    new_blaze_dic["Left_Foot"] = (np.array(blaze_dic["Left_heel"])+np.array(blaze_dic["Left_foot_index"]))/2
    #right leg
    new_blaze_dic["Right_Tigh"] = (np.array(blaze_dic["Right_hip"])+np.array(blaze_dic["Right_knee"]))/2
    new_blaze_dic["Right_Tibia"] = (np.array(blaze_dic["Right_knee"])+np.array(blaze_dic["Right_ankle"]))/2
    new_blaze_dic["Right_Foot"] = (np.array(blaze_dic["Right_heel"])+np.array(blaze_dic["Right_foot_index"]))/2
    return new_blaze_dic

In [16]:
def kinect_frame_to_vicon(kinect_line):
    data = kinect_line.split(" ")[:-1]
    # print(kinect)
    new_kinect_dic = {}
    #spine
    new_kinect_dic["Head"] = np.array([float(data[3*7]), float(data[3*7+1]), float(data[3*7+2])])
    new_kinect_dic["Chest"] = (np.array([float(data[1*7]), float(data[1*7+1]), float(data[1*7+2])]) + np.array([float(data[20*7]), float(data[20*7+1]), float(data[20*7+2])]))/2
    new_kinect_dic["Hips"] = np.array([float(data[0*7]), float(data[0*7+1]), float(data[0*7+2])])
    #left arm
    new_kinect_dic["Left_Shoulder"] = np.array([float(data[4*7]), float(data[4*7+1]), float(data[4*7+2])])
    new_kinect_dic["Left_Arm"] = (np.array([float(data[4*7]), float(data[4*7+1]), float(data[4*7+2])])+np.array([float(data[5*7]), float(data[5*7+1]), float(data[5*7+2])]))/2
    new_kinect_dic["Left_Forearm"] = (np.array([float(data[5*7]), float(data[5*7+1]), float(data[5*7+2])])+np.array([float(data[6*7]), float(data[6*7+1]), float(data[6*7+2])]))/2
    new_kinect_dic["Left_Hand"] = np.array([float(data[7*7]), float(data[7*7+1]), float(data[7*7+2])])
    #right arm
    new_kinect_dic["Right_Shoulder"] = np.array([float(data[8*7]), float(data[8*7+1]), float(data[8*7+2])])
    new_kinect_dic["Right_Arm"] = (np.array([float(data[8*7]), float(data[8*7+1]), float(data[8*7+2])])+np.array([float(data[9*7]), float(data[9*7+1]), float(data[9*7+2])]))/2
    new_kinect_dic["Right_Forearm"] = (np.array([float(data[9*7]), float(data[9*7+1]), float(data[9*7+2])])+np.array([float(data[10*7]), float(data[10*7+1]), float(data[10*7+2])]))/2
    new_kinect_dic["Right_Hand"] = np.array([float(data[11*7]), float(data[11*7+1]), float(data[11*7+2])])
    #left leg
    new_kinect_dic["Left_Tigh"] = (np.array([float(data[12*7]), float(data[12*7+1]), float(data[12*7+2])])+np.array([float(data[13*7]), float(data[13*7+1]), float(data[13*7+2])]))/2
    new_kinect_dic["Left_Tibia"] = (np.array([float(data[13*7]), float(data[13*7+1]), float(data[13*7+2])])+np.array([float(data[14*7]), float(data[14*7+1]), float(data[14*7+2])]))/2
    new_kinect_dic["Left_Foot"] = np.array([float(data[15*7]), float(data[15*7+1]), float(data[15*7+2])])
    #right leg
    new_kinect_dic["Right_Tigh"] = (np.array([float(data[16*7]), float(data[16*7+1]), float(data[16*7+2])])+np.array([float(data[17*7]), float(data[17*7+1]), float(data[17*7+2])]))/2
    new_kinect_dic["Right_Tibia"] = (np.array([float(data[17*7]), float(data[17*7+1]), float(data[17*7+2])])+np.array([float(data[18*7]), float(data[18*7+1]), float(data[18*7+2])]))/2
    new_kinect_dic["Right_Foot"] = np.array([float(data[19*7]), float(data[19*7+1]), float(data[19*7+2])])
    return new_kinect_dic

In [17]:
# dic = document_path_list("..\data")
# list_openpose = dic['openpose']
# list_try = ["1","2"]
# print(type(list_try))
# st = dic['openpose'][0]
# print(st)
# with open(st,'r+', encoding='utf-8') as f:
#     print(json.load(f))
# f.close()

In [18]:
aDic = document_path_list("..\data")
num = 5
open_list = aDic['openpose']
blaze_list = aDic["blazepose"]

# openpose
open_file = open(open_list[num],'r+')
open_dic = json.loads(open_file.read())
open_file.close()
# print(open_dic["positions"]['1.0'])
new_open = open_frame_to_vicon(open_dic["positions"]['1.0'])
print("openpose: {}\n".format(new_open))

#blazepose
blaze_file = open(blaze_list[num],'r+')
blaze_dic = json.loads(blaze_file.read())
blaze_file.close()
blaze = blaze_frame_to_vicon(blaze_dic["positions"]['1.0'])
print("blazepose: {}\n".format(blaze))

#kinect
kinect_file = open(aDic['kinect'][num],'r+')
kinect_data = kinect_file.readlines()
kinect_file.close()
kinect = kinect_frame_to_vicon(kinect_data[0])
print("kinect: {}\n".format(kinect))

#vicon
# print(aDic['vicon'])
# del dict
vicon_file =  open(aDic['vicon'][num], "r+")
vicon_data = vicon_file.readlines()
vicon_file.close()
vicon = vicon_frame_to_dic(vicon_data[10])
print("vicon: {}\n".format(vicon))


openpose: {'Head': array([0.41920732, 0.50271739]), 'Chest': array([0.42682927, 0.62024457]), 'Hips': array([0.42682927, 0.6861413 ]), 'Left_Shoulder': array([0.45731707, 0.55163043]), 'Left_Arm': array([0.46189024, 0.5951087 ]), 'Left_Forearm': array([0.46112805, 0.67119565]), 'Left_Hand': array([nan, nan]), 'Right_Shoulder': array([0.39634146, 0.55434783]), 'Right_Arm': array([0.39557927, 0.59918478]), 'Right_Forearm': array([0.40167683, 0.68070652]), 'Right_Hand': array([nan, nan]), 'Left_Tigh': array([0.46189024, 0.72282609]), 'Left_Tibia': array([0.47027439, 0.8125    ]), 'Left_Foot': array([nan, nan]), 'Right_Tigh': array([0.39481707, 0.72690217]), 'Right_Tibia': array([0.38948171, 0.81521739]), 'Right_Foot': array([nan, nan])}

blazepose: {'Head': array([ 0.57793492,  0.50837761, -0.2771512 ]), 'Chest': array([ 0.56934634,  0.57551008, -0.08394385]), 'Hips': array([5.68022549e-01, 6.09347373e-01, 2.81671528e-05]), 'Left_Shoulder': array([ 0.59442955,  0.54049295, -0.15363856]), 

# Convert to the same reference

In [19]:
def to_blaze_form(a_point, point_ref, point_ref_blaze, dis_ref, dis_ref_blaze):
    dis = np.linalg.norm(a_point - point_ref)
#     print("distance: ", dis)
#     print("distance reference", dis_ref)
    dis_blaze = dis/dis_ref*dis_ref_blaze
    return (a_point - point_ref)/dis*dis_blaze + point_ref_blaze

In [20]:
def convert_to_blaze_reference(open_dic, blaze_dic, openpose = False):
    if openpose:
        dis_ref_open = np.linalg.norm(open_dic["Chest"]-open_dic["Hips"])
        dis_ref_blaze = np.linalg.norm(blaze_dic["Chest"][:-1]-blaze_dic["Hips"][:-1])
        point_ref_open = open_dic["Hips"]
        point_ref_blaze = blaze_dic["Hips"][:-1]
    else:
        dis_ref_open = np.linalg.norm(open_dic["Chest"]-open_dic["Hips"])
        dis_ref_blaze = np.linalg.norm(blaze_dic["Chest"]-blaze_dic["Hips"])
    #     dis_ref_kin = np.linalg.norm(kinect_dic["Chest"]-kinect_dic["Hips"])
    #     dis_ref_vicon = np.linalg.norm(vicon_dic["Chest"]-vicon_dic["Hips"])

        point_ref_open = open_dic["Hips"]
        point_ref_blaze = blaze_dic["Hips"]
    #     point_ref_kin = kinect_dic["Hips"]
    #     point_ref_vicon = vicon_dic["Hips"]
    a_dic = {}
    for key, item in open_dic.items():
#         print(key)
        a_dic[key] = to_blaze_form(item, point_ref_open, point_ref_blaze, dis_ref_open, dis_ref_blaze)
    return a_dic
    

In [22]:
# print(convert_to_blaze_reference(new_open,blaze,openpose = True))
print(convert_to_blaze_reference(kinect,blaze))
# print(convert_to_blaze_reference(vicon,blaze))

{'Head': array([ 0.56903829,  0.83813152, -0.0483807 ]), 'Chest': array([ 0.56928823,  0.69910471, -0.0118054 ]), 'Hips': array([nan, nan, nan]), 'Left_Shoulder': array([ 0.5094603 ,  0.76204488, -0.01746135]), 'Left_Arm': array([ 0.49732811,  0.71969139, -0.01789347]), 'Left_Forearm': array([ 0.48973587,  0.64797485, -0.04673848]), 'Left_Hand': array([ 0.49691578,  0.60476529, -0.09387713]), 'Right_Shoulder': array([ 0.63066737,  0.7595444 , -0.03391914]), 'Right_Arm': array([ 0.63607992,  0.71783193, -0.04143037]), 'Right_Forearm': array([ 0.62144126,  0.64250473, -0.07727983]), 'Right_Hand': array([ 0.58501037,  0.5869539 , -0.12385448]), 'Left_Tigh': array([ 0.51081916,  0.598064  , -0.0553283 ]), 'Left_Tibia': array([ 0.48670518,  0.51086581, -0.0911298 ]), 'Left_Foot': array([ 0.47508395,  0.41966261, -0.09187508]), 'Right_Tigh': array([ 0.60721548,  0.59494485, -0.07174159]), 'Right_Tibia': array([ 0.61797845,  0.50556915, -0.10439252]), 'Right_Foot': array([ 0.63080288,  0.4185

C:\Users\22628\AppData\Local\Temp\ipykernel_13424\106319807.py:6: RuntimeWarning: invalid value encountered in true_divide
  return (a_point - point_ref)/dis*dis_blaze + point_ref_blaze


# Error for one video

In [34]:
def comparaison_onevideo(open_path, blaze_path, kinect_path, vicon_path):
    print(open_path)
    print(blaze_path)
    print(kinect_path)
    print(vicon_path)
    print()
    
    # openpose
    open_file = open(open_path,'r+')
    open_dic = json.loads(open_file.read())
    open_file.close()
    open_all_frame = open_dic['positions']
    
    #blazepose
    blaze_file = open(blaze_path,'r+')
    blaze_dic = json.loads(blaze_file.read())
    blaze_file.close()
    blaze_all_frame = blaze_dic['positions']
    
    #kinect
    kinect_file = open(kinect_path,'r+')
    kinect_all_frame = kinect_file.readlines()
    kinect_file.close()

    #vicon
    vicon_file =  open(vicon_path, "r+")
    vicon_all_frame = vicon_file.readlines()
    vicon_file.close()
    
    frame_min = min(len(open_all_frame.keys()),len(blaze_all_frame.keys()),len(kinect_all_frame), len(vicon_all_frame))
    list_sklt = ["Right_Forearm", "Left_Forearm", "Right_Arm", "Left_Arm", "Chest",
                    "Right_Tigh", "Left_Tigh", "Right_Shoulder", "Left_Shoulder",
                    "Right_Hand", "Left_Hand", "Right_Foot", "Left_Foot",
                    "Hips", "Head", "Right_Tibia", "Left_Tibia"]
    Error_blazeVicon_video = {}
    Error_kintVicon_video = {}
    Error_openVicon_video = {}
    for sklt in list_sklt:
        Error_blazeVicon_video[sklt] = np.zeros(frame_min)
        Error_kintVicon_video[sklt] = np.zeros(frame_min)
        Error_openVicon_video[sklt] = np.zeros(frame_min)
#         print("k: ", len_kin,", b: ", len_bla)
    for frame in range(frame_min):
        blaze_sklt = blaze_frame_to_vicon(blaze_all_frame[str(frame+1)+".0"])
        open_temp = open_frame_to_vicon(open_all_frame[str(frame+1)+".0"])
        kin_temp = kinect_frame_to_vicon(kinect_all_frame[frame])
        vicon_temp = vicon_frame_to_dic(vicon_all_frame[frame])
        
        open_sklt = convert_to_blaze_reference(open_temp, blaze_sklt, openpose = True)
        vicon_sklt = convert_to_blaze_reference(vicon_temp, blaze_sklt)
        kin_sklt = convert_to_blaze_reference(kin_temp, blaze_sklt)
#         print(blaze_sklt)
#         print(kin_sklt)
#         print("frame yesyes : ", frame)
        for ske in list_sklt:
            Error_blazeVicon_video[ske][frame] = np.linalg.norm(blaze_sklt[ske] - vicon_sklt[ske])
            Error_kintVicon_video[ske][frame] = np.linalg.norm(kin_sklt[ske] - vicon_sklt[ske])
            Error_openVicon_video[ske][frame] = np.linalg.norm(open_sklt[ske] - vicon_sklt[ske][:-1])
    return  Error_openVicon_video, Error_blazeVicon_video, Error_kintVicon_video

In [35]:
# aDic = document_path_list("..\data")
# num = 0
# comparaison_onevideo(aDic["openpose"][num], aDic["blazepose"][num], aDic["kinect"][num], aDic["vicon"][num])

# Error for all video

In [39]:
def get_error_dic(dic):
    Error_dic = {}
    list_sklt = ["Right_Forearm", "Left_Forearm", "Right_Arm", "Left_Arm", "Chest",
                    "Right_Tigh", "Left_Tigh", "Right_Shoulder", "Left_Shoulder",
                    "Right_Hand", "Left_Hand", "Right_Foot", "Left_Foot",
                    "Hips", "Head", "Right_Tibia", "Left_Tibia"]
    list_error = ['Error_blazeVicon', 'Error_kintVicon', 'Error_openVicon']

    video_total_len = len(dic["blazepose"])
    for error_type in list_error:
        Error_dic[error_type] = {}
        for sklt in list_sklt:
            Error_dic[error_type][sklt] = np.zeros(video_total_len)
    for i in range(video_total_len):
        Error_openVicon_one, Error_blazeVicon_one, Error_kinVicon_one  = comparaison_onevideo(dic["openpose"][i],dic["blazepose"][i],dic["kinect"][i], dic["vicon"][i])
        for skl in list_sklt:
            Error_dic["Error_openVicon"][skl][i] = np.nanmean(Error_openVicon_one[skl])
            Error_dic["Error_blazeVicon"][skl][i] = np.nanmean(Error_blazeVicon_one[skl])
            Error_dic["Error_kintVicon"][skl][i] = np.nanmean(Error_kinVicon_one[skl])
#             print("num: {}, joint: {}, error:{}, {}, {}".format(i, skl, np.nanmean(Error_openVicon_one[skl]), np.nanmean(Error_blazeVicon_one[skl]), np.nanmean(Error_kinVicon_one[skl])))
    return Error_dic

In [40]:
aDic = document_path_list("..\data")
Error_dic = get_error_dic(aDic)


re = "../results/pkls"
if not os.path.exists(re):
    os.mkdir(re)
Error_path = os.path.join(re,"Error_compare_Vicon.pkl")
with open(Error_path,'wb') as f:
    pkl.dump(Error_dic, f)

..\data\group3\openpose\G3-Anon-CTK-P1T1-Unknown-C-0.json
..\data\group3\blazepose\G3-Faces-CTK-P1T1-Unknown-C-0.json
..\data\group3\kinect\G3-Kinect-CTK-P1T1-Unknown-C-0.txt
..\data\group3\vicon\G3-Vicon-CTK-P1T1-Unknown-C-0.txt

..\data\group3\openpose\G3-Anon-CTK-P1T1-Unknown-C-1.json
..\data\group3\blazepose\G3-Faces-CTK-P1T1-Unknown-C-1.json
..\data\group3\kinect\G3-Kinect-CTK-P1T1-Unknown-C-1.txt
..\data\group3\vicon\G3-Vicon-CTK-P1T1-Unknown-C-1.txt



C:\Users\22628\AppData\Local\Temp\ipykernel_13424\106319807.py:6: RuntimeWarning: invalid value encountered in true_divide
  return (a_point - point_ref)/dis*dis_blaze + point_ref_blaze
C:\Users\22628\AppData\Local\Temp\ipykernel_13424\957658817.py:17: RuntimeWarning: Mean of empty slice
  Error_dic["Error_openVicon"][skl][i] = np.nanmean(Error_openVicon_one[skl])
C:\Users\22628\AppData\Local\Temp\ipykernel_13424\957658817.py:18: RuntimeWarning: Mean of empty slice
  Error_dic["Error_blazeVicon"][skl][i] = np.nanmean(Error_blazeVicon_one[skl])
C:\Users\22628\AppData\Local\Temp\ipykernel_13424\957658817.py:19: RuntimeWarning: Mean of empty slice
  Error_dic["Error_kintVicon"][skl][i] = np.nanmean(Error_kinVicon_one[skl])


..\data\group3\openpose\G3-Anon-CTK-P1T1-Unknown-C-2.json
..\data\group3\blazepose\G3-Faces-CTK-P1T1-Unknown-C-2.json
..\data\group3\kinect\G3-Kinect-CTK-P1T1-Unknown-C-2.txt
..\data\group3\vicon\G3-Vicon-CTK-P1T1-Unknown-C-2.txt

..\data\group3\openpose\G3-Anon-CTK-P1T1-Unknown-C-3.json
..\data\group3\blazepose\G3-Faces-CTK-P1T1-Unknown-C-3.json
..\data\group3\kinect\G3-Kinect-CTK-P1T1-Unknown-C-3.txt
..\data\group3\vicon\G3-Vicon-CTK-P1T1-Unknown-C-3.txt

..\data\group3\openpose\G3-Anon-CTK-P1T1-Unknown-C-4.json
..\data\group3\blazepose\G3-Faces-CTK-P1T1-Unknown-C-4.json
..\data\group3\kinect\G3-Kinect-CTK-P1T1-Unknown-C-4.txt
..\data\group3\vicon\G3-Vicon-CTK-P1T1-Unknown-C-4.txt

..\data\group3\openpose\G3-Anon-CTK-P1T1-Unknown-E1B1-0.json
..\data\group3\blazepose\G3-Faces-CTK-P1T1-Unknown-E1B1-0.json
..\data\group3\kinect\G3-Kinect-CTK-P1T1-Unknown-E1B1-0.txt
..\data\group3\vicon\G3-Vicon-CTK-P1T1-Unknown-E1B1-0.txt

..\data\group3\openpose\G3-Anon-CTK-P1T1-Unknown-E1B1-1.json
..\

..\data\group3\openpose\G3-Anon-CTK-P1T2-Unknown-E2B3-2.json
..\data\group3\blazepose\G3-Faces-CTK-P1T2-Unknown-E2B3-2.json
..\data\group3\kinect\G3-Kinect-CTK-P1T2-Unknown-E2B3-2.txt
..\data\group3\vicon\G3-Vicon-CTK-P1T2-Unknown-E2B3-2.txt

..\data\group3\openpose\G3-Anon-CTK-P1T2-Unknown-E2B3-3.json
..\data\group3\blazepose\G3-Faces-CTK-P1T2-Unknown-E2B3-3.json
..\data\group3\kinect\G3-Kinect-CTK-P1T2-Unknown-E2B3-3.txt
..\data\group3\vicon\G3-Vicon-CTK-P1T2-Unknown-E2B3-3.txt

..\data\group3\openpose\G3-Anon-CTK-P1T2-Unknown-E2B3-4.json
..\data\group3\blazepose\G3-Faces-CTK-P1T2-Unknown-E2B3-4.json
..\data\group3\kinect\G3-Kinect-CTK-P1T2-Unknown-E2B3-4.txt
..\data\group3\vicon\G3-Vicon-CTK-P1T2-Unknown-E2B3-4.txt

..\data\group3\openpose\G3-Anon-CTK-P1T3-Unknown-C-0.json
..\data\group3\blazepose\G3-Faces-CTK-P1T3-Unknown-C-0.json
..\data\group3\kinect\G3-Kinect-CTK-P1T3-Unknown-C-0.txt
..\data\group3\vicon\G3-Vicon-CTK-P1T3-Unknown-C-0.txt

..\data\group3\openpose\G3-Anon-CTK-P1T3

..\data\group3\openpose\G3-Anon-CTK-P2T1-Unknown-E1B2-2.json
..\data\group3\blazepose\G3-Faces-CTK-P2T1-Unknown-E1B2-2.json
..\data\group3\kinect\G3-Kinect-CTK-P2T1-Unknown-E1B2-2.txt
..\data\group3\vicon\G3-Vicon-CTK-P2T1-Unknown-E1B2-2.txt

..\data\group3\openpose\G3-Anon-CTK-P2T1-Unknown-E1B2-3.json
..\data\group3\blazepose\G3-Faces-CTK-P2T1-Unknown-E1B2-3.json
..\data\group3\kinect\G3-Kinect-CTK-P2T1-Unknown-E1B2-3.txt
..\data\group3\vicon\G3-Vicon-CTK-P2T1-Unknown-E1B2-3.txt

..\data\group3\openpose\G3-Anon-CTK-P2T1-Unknown-E1B2-4.json
..\data\group3\blazepose\G3-Faces-CTK-P2T1-Unknown-E1B2-4.json
..\data\group3\kinect\G3-Kinect-CTK-P2T1-Unknown-E1B2-4.txt
..\data\group3\vicon\G3-Vicon-CTK-P2T1-Unknown-E1B2-4.txt

..\data\group3\openpose\G3-Anon-CTK-P2T1-Unknown-E1B3-0.json
..\data\group3\blazepose\G3-Faces-CTK-P2T1-Unknown-E1B3-0.json
..\data\group3\kinect\G3-Kinect-CTK-P2T1-Unknown-E1B3-0.txt
..\data\group3\vicon\G3-Vicon-CTK-P2T1-Unknown-E1B3-0.txt

..\data\group3\openpose\G3-A

...

..\data\group3\openpose\G3-Anon-RTK-P1T2-Unknown-E2B1-2.json
..\data\group3\blazepose\G3-Faces-RTK-P1T2-Unknown-E2B1-2.json
..\data\group3\kinect\G3-Kinect-RTK-P1T2-Unknown-E2B1-2.txt
..\data\group3\vicon\G3-Vicon-RTK-P1T2-Unknown-E2B1-2.txt

..\data\group3\openpose\G3-Anon-RTK-P1T2-Unknown-E2B1-3.json
..\data\group3\blazepose\G3-Faces-RTK-P1T2-Unknown-E2B1-3.json
..\data\group3\kinect\G3-Kinect-RTK-P1T2-Unknown-E2B1-3.txt
..\data\group3\vicon\G3-Vicon-RTK-P1T2-Unknown-E2B1-3.txt

..\data\group3\openpose\G3-Anon-RTK-P1T2-Unknown-E2B1-4.json
..\data\group3\blazepose\G3-Faces-RTK-P1T2-Unknown-E2B1-4.json
..\data\group3\kinect\G3-Kinect-RTK-P1T2-Unknown-E2B1-4.txt
..\data\group3\vicon\G3-Vicon-RTK-P1T2-Unknown-E2B1-4.txt

..\data\group3\openpose\G3-Anon-RTK-P1T2-Unknown-E2B2-0.json
..\data\group3\blazepose\G3-Faces-RTK-P1T2-Unknown-E2B2-0.json
..\data\group3\kinect\G3-Kinect-RTK-P1T2-Unknown-E2B2-0.txt
..\data\group3\vicon\G3-Vicon-RTK-P1T2-Unknown-E2B2-0.txt

..\data\group3\openpose\G3-A

..\data\group3\openpose\G3-Anon-RTK-P2T1-Unknown-C-2.json
..\data\group3\blazepose\G3-Faces-RTK-P2T1-Unknown-C-2.json
..\data\group3\kinect\G3-Kinect-RTK-P2T1-Unknown-C-2.txt
..\data\group3\vicon\G3-Vicon-RTK-P2T1-Unknown-C-2.txt

..\data\group3\openpose\G3-Anon-RTK-P2T1-Unknown-C-3.json
..\data\group3\blazepose\G3-Faces-RTK-P2T1-Unknown-C-3.json
..\data\group3\kinect\G3-Kinect-RTK-P2T1-Unknown-C-3.txt
..\data\group3\vicon\G3-Vicon-RTK-P2T1-Unknown-C-3.txt

..\data\group3\openpose\G3-Anon-RTK-P2T1-Unknown-C-4.json
..\data\group3\blazepose\G3-Faces-RTK-P2T1-Unknown-C-4.json
..\data\group3\kinect\G3-Kinect-RTK-P2T1-Unknown-C-4.txt
..\data\group3\vicon\G3-Vicon-RTK-P2T1-Unknown-C-4.txt

..\data\group3\openpose\G3-Anon-RTK-P2T1-Unknown-E1B1-0.json
..\data\group3\blazepose\G3-Faces-RTK-P2T1-Unknown-E1B1-0.json
..\data\group3\kinect\G3-Kinect-RTK-P2T1-Unknown-E1B1-0.txt
..\data\group3\vicon\G3-Vicon-RTK-P2T1-Unknown-E1B1-0.txt

..\data\group3\openpose\G3-Anon-RTK-P2T1-Unknown-E1B1-1.json
..\

..\data\group3\openpose\G3-Anon-RTK-P2T2-Unknown-E2B3-2.json
..\data\group3\blazepose\G3-Faces-RTK-P2T2-Unknown-E2B3-2.json
..\data\group3\kinect\G3-Kinect-RTK-P2T2-Unknown-E2B3-2.txt
..\data\group3\vicon\G3-Vicon-RTK-P2T2-Unknown-E2B3-2.txt

..\data\group3\openpose\G3-Anon-RTK-P2T2-Unknown-E2B3-3.json
..\data\group3\blazepose\G3-Faces-RTK-P2T2-Unknown-E2B3-3.json
..\data\group3\kinect\G3-Kinect-RTK-P2T2-Unknown-E2B3-3.txt
..\data\group3\vicon\G3-Vicon-RTK-P2T2-Unknown-E2B3-3.txt

..\data\group3\openpose\G3-Anon-RTK-P2T2-Unknown-E2B3-4.json
..\data\group3\blazepose\G3-Faces-RTK-P2T2-Unknown-E2B3-4.json
..\data\group3\kinect\G3-Kinect-RTK-P2T2-Unknown-E2B3-4.txt
..\data\group3\vicon\G3-Vicon-RTK-P2T2-Unknown-E2B3-4.txt

..\data\group3\openpose\G3-Anon-RTK-P2T3-Unknown-C-0.json
..\data\group3\blazepose\G3-Faces-RTK-P2T3-Unknown-C-0.json
..\data\group3\kinect\G3-Kinect-RTK-P2T3-Unknown-C-0.txt
..\data\group3\vicon\G3-Vicon-RTK-P2T3-Unknown-C-0.txt

..\data\group3\openpose\G3-Anon-RTK-P2T3

..\data\group3\openpose\G3-Anon-RTK-P3T1-Unknown-E1B2-2.json
..\data\group3\blazepose\G3-Faces-RTK-P3T1-Unknown-E1B2-2.json
..\data\group3\kinect\G3-Kinect-RTK-P3T1-Unknown-E1B2-2.txt
..\data\group3\vicon\G3-Vicon-RTK-P3T1-Unknown-E1B2-2.txt

..\data\group3\openpose\G3-Anon-RTK-P3T1-Unknown-E1B2-3.json
..\data\group3\blazepose\G3-Faces-RTK-P3T1-Unknown-E1B2-3.json
..\data\group3\kinect\G3-Kinect-RTK-P3T1-Unknown-E1B2-3.txt
..\data\group3\vicon\G3-Vicon-RTK-P3T1-Unknown-E1B2-3.txt

..\data\group3\openpose\G3-Anon-RTK-P3T1-Unknown-E1B2-4.json
..\data\group3\blazepose\G3-Faces-RTK-P3T1-Unknown-E1B2-4.json
..\data\group3\kinect\G3-Kinect-RTK-P3T1-Unknown-E1B2-4.txt
..\data\group3\vicon\G3-Vicon-RTK-P3T1-Unknown-E1B2-4.txt

..\data\group3\openpose\G3-Anon-RTK-P3T1-Unknown-E1B3-0.json
..\data\group3\blazepose\G3-Faces-RTK-P3T1-Unknown-E1B3-0.json
..\data\group3\kinect\G3-Kinect-RTK-P3T1-Unknown-E1B3-0.txt
..\data\group3\vicon\G3-Vicon-RTK-P3T1-Unknown-E1B3-0.txt

..\data\group3\openpose\G3-A

..\data\group3\openpose\G3-Anon-RTK-P3T3-Unknown-E3B1-2.json
..\data\group3\blazepose\G3-Faces-RTK-P3T3-Unknown-E3B1-2.json
..\data\group3\kinect\G3-Kinect-RTK-P3T3-Unknown-E3B1-2.txt
..\data\group3\vicon\G3-Vicon-RTK-P3T3-Unknown-E3B1-2.txt

..\data\group3\openpose\G3-Anon-RTK-P3T3-Unknown-E3B1-3.json
..\data\group3\blazepose\G3-Faces-RTK-P3T3-Unknown-E3B1-3.json
..\data\group3\kinect\G3-Kinect-RTK-P3T3-Unknown-E3B1-3.txt
..\data\group3\vicon\G3-Vicon-RTK-P3T3-Unknown-E3B1-3.txt

..\data\group3\openpose\G3-Anon-RTK-P3T3-Unknown-E3B1-4.json
..\data\group3\blazepose\G3-Faces-RTK-P3T3-Unknown-E3B1-4.json
..\data\group3\kinect\G3-Kinect-RTK-P3T3-Unknown-E3B1-4.txt
..\data\group3\vicon\G3-Vicon-RTK-P3T3-Unknown-E3B1-4.txt

..\data\group3\openpose\G3-Anon-RTK-P3T3-Unknown-E3B2-0.json
..\data\group3\blazepose\G3-Faces-RTK-P3T3-Unknown-E3B2-0.json
..\data\group3\kinect\G3-Kinect-RTK-P3T3-Unknown-E3B2-0.txt
..\data\group3\vicon\G3-Vicon-RTK-P3T3-Unknown-E3B2-0.txt

..\data\group3\openpose\G3-A

In [44]:
# print(Error_dic)

# Error_mean_std

In [42]:
error_path = "../results/pkls/Error_compare_Vicon.pkl"
with open(error_path, "rb") as f:
    Error_all = pkl.load(f)
f.close()

In [45]:
# print(Error_all)

In [46]:
def get_error_mean_std(Error_all):
#     group_mean = ['group1A', 'group2A']
    Error_mean = {}
    Error_std = {}
    for error_type in Error_all.keys():
        Error_mean[error_type] = {}
        Error_std[error_type] = {}
        for sklt in Error_all[error_type].keys():
            temp_arr = Error_all[error_type][sklt]
            Error_mean[error_type][sklt] = np.nanmean(np.where(temp_arr!=0,temp_arr,np.nan))
            Error_std[error_type][sklt] = np.nanstd(np.where(temp_arr!=0,temp_arr,np.nan))
#         print(Error_mean[error_type].values())
        Error_mean[error_type]["average"] = np.nanmean(list(Error_mean[error_type].values()))
        Error_std[error_type]["average"] = np.nanmean(list(Error_std[error_type].values()))
    return Error_mean, Error_std

In [47]:
mean,std = get_error_mean_std(Error_all)

C:\Users\22628\AppData\Local\Temp\ipykernel_13424\1422381920.py:10: RuntimeWarning: Mean of empty slice
  Error_mean[error_type][sklt] = np.nanmean(np.where(temp_arr!=0,temp_arr,np.nan))


In [48]:
Error_mean_path = "../results/pkls/Error_Vicon_mean.pkl"
with open(Error_mean_path,'wb') as f:
    pkl.dump(mean, f)
f.close()

Error_std_path = "../results/pkls/Error_Vicon_std.pkl"
with open(Error_std_path,'wb') as f:
    pkl.dump(std, f)
f.close()